# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [2]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [4]:
# Run this code:
salaries = pd.read_csv('../Data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [5]:
# Your code here
print(salaries.head())

                  Name                              Job Titles  \
0    AARON,  JEFFERY M                                SERGEANT   
1      AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2  AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3  ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4    ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   

         Department Full or Part-Time Salary or Hourly  Typical Hours  \
0            POLICE                 F           Salary            NaN   
1            POLICE                 F           Salary            NaN   
2  GENERAL SERVICES                 F           Salary            NaN   
3       WATER MGMNT                 F           Salary            NaN   
4              OEMC                 P           Hourly           20.0   

   Annual Salary  Hourly Rate  
0       101442.0          NaN  
1        94122.0          NaN  
2       101592.0          NaN  
3       110064.0          NaN  
4   

# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [6]:
# Your code here
# Importar la función ttest_1samp desde scipy.stats para la prueba t de una muestra
from scipy.stats import ttest_1samp

# Filtrar los datos para obtener solo los trabajadores por hora y que tienen salario por hora definido
hourly_workers = salaries[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Hourly Rate'].notna())]

# Realizar la prueba de hipótesis para una muestra
# Hipótesis nula: El salario por hora promedio es igual a $30
# Nivel de confianza del 95% (alpha = 0.05)
t_statistic, p_value = ttest_1samp(hourly_workers['Hourly Rate'], 30)

# Imprimir los resultados
print(f"T-statistic: {t_statistic:.3f}")
print(f"P-value: {p_value:.3f}")

# Interpretar el resultado
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: El salario por hora es significativamente diferente de $30/hora.")
else:
    print("No se puede rechazar la hipótesis nula: No hay suficiente evidencia para decir que el salario por hora es diferente de $30/hora.")

T-statistic: 20.620
P-value: 0.000
Se rechaza la hipótesis nula: El salario por hora es significativamente diferente de $30/hora.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [7]:
# Your code here
# Importar la función t para calcular intervalos de confianza
from scipy.stats import t

# Calcular la media y el error estándar del salario por hora
mean_hourly_rate = hourly_workers['Hourly Rate'].mean()
std_error_hourly_rate = hourly_workers['Hourly Rate'].sem()

# Definir los grados de libertad
# Usamos n-1 para los grados de libertad donde n es el número de observaciones
degrees_freedom = len(hourly_workers) - 1

# Calcular el intervalo de confianza del 95% para la media del salario por hora
confidence_level = 0.95
lower_bound, upper_bound = t.interval(confidence_level, df=degrees_freedom, loc=mean_hourly_rate, scale=std_error_hourly_rate)

# Imprimir los resultados
print(f"Intervalo de Confianza del 95% para el Salario por Hora Medio:")
print(f"Lower Bound: {lower_bound:.2f}")
print(f"Upper Bound: {upper_bound:.2f}")

Intervalo de Confianza del 95% para el Salario por Hora Medio:
Lower Bound: 32.52
Upper Bound: 33.05


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [8]:
# Your code here
# Importar la función proportions_ztest de statsmodels para la prueba de proporciones
from statsmodels.stats.proportion import proportions_ztest

# Contar el número de trabajadores por hora y el número total de empleados
num_hourly_workers = len(hourly_workers)
total_employees = len(salaries)

# Definir la hipótesis nula y el nivel de confianza
# Hipótesis nula: La proporción de trabajadores por hora es igual al 25%
# Nivel de confianza del 95%
h0_proportion = 0.25

# Realizar la prueba de proporciones
z_statistic, p_value = proportions_ztest(num_hourly_workers, total_employees, h0_proportion)

# Imprimir los resultados
print(f"Z-statistic: {z_statistic:.3f}")
print(f"P-value: {p_value:.3f}")

# Interpretar el resultado
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: La proporción de trabajadores por hora es significativamente diferente del 25%.")
else:
    print("No se puede rechazar la hipótesis nula: No hay suficiente evidencia para decir que la proporción de trabajadores por hora es diferente del 25%.")

Z-statistic: -3.510
P-value: 0.000
Se rechaza la hipótesis nula: La proporción de trabajadores por hora es significativamente diferente del 25%.
